# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
#set csv file
weather_csv = "../output_data/cities.csv"

#read csv into dataframe
weather_df = pd.read_csv(weather_csv)

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saskylakh,71.9167,114.0833,-22.32,84,0,6.64,RU,1613012657
1,lucapa,-8.4192,20.7447,64.81,96,100,2.68,AO,1613012651
2,palmer,42.1584,-72.3287,16.00,73,90,1.83,US,1613012060
3,vardo,70.3705,31.1107,24.33,86,100,13.35,NO,1613012760
4,puerto ayora,-0.7393,-90.3518,75.00,99,79,1.01,EC,1613012762
...,...,...,...,...,...,...,...,...,...
573,verkhnevilyuysk,63.4453,120.3167,-15.57,88,93,2.66,RU,1613013465
574,camacari,-12.6975,-38.3242,82.40,69,40,11.50,BR,1613013466
575,bayburt,40.2589,40.2278,22.73,93,2,1.90,TR,1613013467
576,paris,48.8534,2.3488,23.00,80,0,10.36,FR,1613012930


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store humidity and convert to float
humidity = weather_df["Humidity"].astype(float)

In [58]:
# Plot Heatmap
fig = gmaps.figure(center=(0,0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
vacay_weather_df = weather_df.loc[(weather_df["Max Temp"]>=65) & (weather_df["Max Temp"]<=90) & (weather_df["Wind Speed"]<=5) & (weather_df["Cloudiness"]==0), :]
vacay_weather_df = vacay_weather_df.dropna()
vacay_weather_df = vacay_weather_df.head(10)

vacay_weather_df.reset_index(inplace=True)
vacay_weather_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,35,saint-pierre,-21.3393,55.4781,77.00,69,0,4.61,RE,1613012435
1,135,chandbali,20.7833,86.7667,72.03,61,0,4.74,IN,1613012920
2,143,phetchabun,16.2500,101.0833,80.01,64,0,2.30,TH,1613012929
3,165,flinders,-34.5833,150.8552,82.00,60,0,4.00,AU,1613012956
4,167,sardarshahr,28.4333,74.4833,65.61,28,0,4.03,IN,1613012824
5,177,tura,25.5198,90.2201,72.97,34,0,2.42,IN,1613012972
6,218,saint-philippe,-21.3585,55.7679,77.00,69,0,4.61,RE,1613013008
7,240,florianopolis,-27.6146,-48.5012,69.80,94,0,0.83,BR,1613013005
8,266,coihaique,-45.5752,-72.0662,69.80,49,0,3.44,CL,1613013086
9,363,imbituba,-28.2400,-48.6703,69.80,94,0,3.49,BR,1613013204


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [52]:
vacay_weather_df["Hotel Name"] = ""

vacay_weather_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,35,saint-pierre,-21.3393,55.4781,77.00,69,0,4.61,RE,1613012435,
1,135,chandbali,20.7833,86.7667,72.03,61,0,4.74,IN,1613012920,
2,143,phetchabun,16.2500,101.0833,80.01,64,0,2.30,TH,1613012929,
3,165,flinders,-34.5833,150.8552,82.00,60,0,4.00,AU,1613012956,
4,167,sardarshahr,28.4333,74.4833,65.61,28,0,4.03,IN,1613012824,
5,177,tura,25.5198,90.2201,72.97,34,0,2.42,IN,1613012972,
6,218,saint-philippe,-21.3585,55.7679,77.00,69,0,4.61,RE,1613013008,
7,240,florianopolis,-27.6146,-48.5012,69.80,94,0,0.83,BR,1613013005,
8,266,coihaique,-45.5752,-72.0662,69.80,49,0,3.44,CL,1613013086,
9,363,imbituba,-28.2400,-48.6703,69.80,94,0,3.49,BR,1613013204,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
